## Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Config notebook

In [2]:
gral_dir = '../../../data/external/santiago/'
project_name = 'redvial2019_buffer_3750m_c_utilidad_2'
p_code = '00'
public_space_quality_dir = gral_dir + f"calidad_ep/{p_code}_{project_name}/{project_name}.shp"

# Projection to be used whenever necessary
projected_crs = 'EPSG:32719'

filtering_column = 'pje_ep'

## Load data

In [21]:
# 1.0 --------------- LOAD DATA
# ------------------- This step loads the public space quality index gdf for the current project
# Load data
pub_space_qty = gpd.read_file(public_space_quality_dir)
# Set CRS
pub_space_qty = pub_space_qty.set_crs(projected_crs)
# Filter for data of relevance
gdf = pub_space_qty[[filtering_column,'geometry']].copy()
gdf = gdf.to_crs("EPSG:4326")

# Show
print(gdf.shape)
gdf.head(1)

(31534, 2)


,pje_ep,geometry
0,0.209171,"LINESTRING (-70.60280 -33.40858, -70.60255 -33..."


In [22]:
#some_lines = gdf.loc[0:10]
# Perform split_lines_with_themselves
#lines_gdf = gdf.copy()
#split_lines_gdf = split_lines_with_themselves(lines_gdf)

# Show
#print(split_lines_gdf.shape)
#split_lines_gdf.head(1)

In [27]:
gdf_exploded = gdf.copy().explode()
gdf_exploded.geom_type.unique()

array(['LineString'], dtype=object)

In [34]:
# split ways
gdf_exploded = gdf.copy().explode()
geometries = [i for i in gdf_exploded.geometry]
ids = [j for j in range(len(geometries))]
unary = gpd.GeoDataFrame({"id": ids, "geometry": geometries}, crs="EPSG:4326")
results = [] #empty list to store the results
for i in range(len(unary)):
    joined = gpd.sjoin(unary.loc[[i]], gdf_exploded, how="inner", op='within')
    results.append(joined)
joined = pd.concat(results)

joined

,id,geometry,index_right0,index_right1,pje_ep
0,0,"LINESTRING (-70.60280 -33.40858, -70.60255 -33...",0,0,0.209171
1,1,"LINESTRING (-70.60036 -33.40805, -70.60032 -33...",1,0,0.273895
2,2,"LINESTRING (-70.60215 -33.40975, -70.60207 -33...",2,0,0.335099
3,3,"LINESTRING (-70.60215 -33.40975, -70.60213 -33...",3,0,0.106013
4,4,"LINESTRING (-70.60426 -33.40751, -70.60418 -33...",4,0,0.019630
...,...,...,...,...,...
31549,31549,"LINESTRING (-70.69190 -33.45897, -70.69192 -33...",31529,0,0.581119
31550,31550,"LINESTRING (-70.65000 -33.48546, -70.64996 -33...",31530,0,0.039703
31551,31551,"LINESTRING (-70.65020 -33.48616, -70.65018 -33...",31531,0,0.012621
31552,31552,"LINESTRING (-70.64985 -33.48548, -70.64968 -33...",31532,0,0.052166


## Step 1 - Building - Split lines with themselves

In [15]:
from shapely.geometry import LineString, MultiLineString
from shapely.ops import split

In [15]:
def split_lines_with_themselves(lines_gdf):
    # Ensure geometries are valid
    lines_gdf = lines_gdf[lines_gdf.is_valid]

    split_lines_list = []

    for idx, line in lines_gdf.iterrows():
        line_to_split = line.geometry  # Initialize the line to be split
        for splitter in lines_gdf.geometry:
            if line_to_split != splitter:  # Avoid splitting a line with itself
                try:
                    # Perform the split
                    split_result = split(line_to_split, splitter)
                    # Ensure that split_result has elements
                    if split_result:
                        split_line = MultiLineString([geom for geom in split_result])
                        
                        # Append the resulting split lines to the list inside the loop
                        if isinstance(split_line, MultiLineString):
                            split_lines_list.extend(split_line.geoms)  # Use .geoms to iterate over the LineStrings in the MultiLineString
                        else:
                            split_lines_list.append(split_line)
                except:
                    continue
    
    # Create a new GeoDataFrame with the split lines
    split_lines_gdf = gpd.GeoDataFrame(geometry=split_lines_list, crs=lines_gdf.crs)
    
    return split_lines_gdf

# Perform split_lines_with_themselves
#lines_gdf = gdf.copy()
#split_lines_gdf = split_lines_with_themselves(lines_gdf)

# Show
#print(split_lines_gdf.shape)
#split_lines_gdf.head(1)